In [1]:
import pdal
import glob
import geopandas as gpd
import numpy as np
import json
import pandas as pd
from shapely import MultiPoint
import alphashape
import shapely

In [2]:
files = glob.glob('downloads/LiDAR/*.laz')

In [3]:
list(files)

['downloads/LiDAR/MDS_3313-143_1000.laz',
 'downloads/LiDAR/MDS_3313-311_1000.laz',
 'downloads/LiDAR/MDS_3313-144_1000.laz',
 'downloads/LiDAR/MDS_3313-312_1000.laz',
 'downloads/LiDAR/san_remo.laz']

In [4]:
!pdal merge {(' ').join(files)} downloads/LiDAR/san_remo.laz

In [5]:
resolution = 0.5

def dem_pipeline(resolution):
    return [
        {
            "type": "readers.las",
            "filename": f'downloads/LiDAR/san_remo.laz',
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"Classification[2:2]"
        },
        {
            "type": "filters.delaunay"
        },
        {
            "type": "filters.faceraster",
            "resolution":resolution,
            # "width": width,
            # "height": height,
            # "origin_x": origin_x,
            # "origin_y": origin_y,
        },        
        {
            "filename":f"results/DEM/MDT-san_remo-50cm.tiff",
            "gdaldriver":"GTiff",
            "type": "writers.raster",
            "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
            "nodata":"0",
            "data_type": "float32",
            # "default_srs": "EPSG:31983"
        }
    ]

def laz_pipeline(resolution):
    return [
        {
            "type":"readers.las",
            "filename":f"downloads/LiDAR/san_remo.laz"
        },
        {
            "filename":f"results/BHM-Z-san_remo.tiff",
            "gdaldriver":"GTiff",
            # "width": width,
            # "height": height,
            # "origin_x": origin_x,
            # "origin_y": origin_y,
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "override_srs": "EPSG:31983",
            "output_type":"max",
            "resolution":resolution,
            "dimension": "Z",
            "data_type": "float32",
            "type": "writers.gdal",
            "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
            "where": "(Classification == 6)",
        },
        {
            "type":"filters.hag_dem",
            "raster": f"results/DEM/MDT-san_remo-50cm.tiff"
        },
        {
            "filename":f"results/BHM-san_remo.tiff",
            "gdaldriver":"GTiff",
            "output_type":"max",
            "resolution": resolution,
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "dimension":"HeightAboveGround",
            # "width": width,
            # "height": height,
            # "origin_x": origin_x,
            # "origin_y": origin_y,
            # "nodata":"0",
            "data_type": "float32",
            "type": "writers.gdal",
            "where": "(Classification == 6)",
            "override_srs": "EPSG:31983"
        },
        {
            "filename":f"results/VHM-san_remo.tiff",
            "gdaldriver":"GTiff",
            "output_type":"max",
            "resolution":resolution,
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "dimension":"HeightAboveGround",
            # "width": width,
            # "height": height,
            # "origin_x": origin_x,
            # "origin_y": origin_y,
            # "nodata":"0",
            "data_type": "float32",
            "type": "writers.gdal",
            "where": "(Classification == 4) || (Classification == 3)",
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"Classification[6:6]"
        },
        {
            "type":"filters.voxeldownsize",
            "cell":0.5,
            "mode":"center"
        },
        {
            "type":"filters.dbscan",
            "min_points":5,
            "eps": (resolution + 0.10) * np.sqrt(2),
            "dimensions":"X,Y,Z"
        },
        {
            "type":"filters.ferry",
            "dimensions":"HeightAboveGround => Z"
        },
        {
            "type":"writers.las",
            "filename":f"results/Cluster-san_remo.laz",
            "extra_dims": "all",
            # "output_dims":"X,Y,Z,ClusterID"
        },
    ]

In [6]:
agg = {
    'coords':list,  
    'Z':['count', 'median', 'sum'], 
    'Intensity':'median', 
    # 'Infrared':'median',
    'Red':'median',
    'Green':'median',
    'Blue':'median'  
}

columns = {
    ('coords', 'list'):'coords',
    ('Z', 'count'):'count',
    ('Z', 'median'):'z_median',
    ('Z', 'sum'):'z_sum',
    ('Intensity', 'median'):'intensity_median',
    # ('Infrared', 'median'):'infrared_median',
    ('Red', 'median'):'red_median',
    ('Green', 'median'):'green_median',
    ('Blue', 'median'):'blue_median',
}

# for _, scm in gdf_articulacao.loc[gdf_articulacao.qmdt_cod.isin(scms)].iterrows():
print('processando')
# coords = [[xy[0], xy[1]] for xy in scm.geometry.exterior.coords]
# xy_max = np.max(np.array(coords), axis=0) 
# xy_min = np.min(np.array(coords), axis=0)
# width_height = np.ceil(xy_max * 2) - np.ceil(xy_min * 2)
# # print(width_height)
# origin_xy = np.floor(xy_min * 2)/2
# print(origin_xy)
dem = dem_pipeline(resolution)

pipeline = pdal.Pipeline(json.dumps(dem))
# pipeline.validate()   
n_points = pipeline.execute()
print(f'Pipeline selected {n_points} points')

laz = laz_pipeline(resolution)

pipeline = pdal.Pipeline(json.dumps(laz))
# pipeline.validate()   
n_points = pipeline.execute()
print(f'Pipeline selected {n_points} points')

arr = pipeline.arrays[0]
df = pd.DataFrame(arr)
df.loc[:, 'coords'] = list(np.dstack([df.X, df.Y])[0])
df.set_index(['X', 'Y']).loc[:, 'Z'] = df.groupby(['X', 'Y']).agg({'Z':'max'})
df.drop_duplicates(subset=['X', 'Y'], keep='last', inplace=True)

df = df[(df.Z > 2.0) & (df.Z < 200.0)].reset_index()

df_agg = df[df.ClusterID > 0].groupby('ClusterID').agg(agg)

df_agg.columns = df_agg.columns.to_flat_index()

df_agg.rename(columns=columns, inplace=True)

df_agg.loc[:, 'geometry'] = df_agg.coords.apply(MultiPoint)

gdf_agg = gpd.GeoDataFrame(df_agg)

gdf_agg.set_crs(epsg=31983, inplace=True)

gdf_agg.drop(columns=['coords']).to_file(f'results/san_remo-multipoint.gpkg', driver='GPKG')




processando


Pipeline selected 25397320 points
Pipeline selected 1866807 points


/var/folders/4v/rx_d3gzj4991_pw5_skhl0980000gn/T/ipykernel_20414/1961080815.py:49: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.set_index(['X', 'Y']).loc[:, 'Z'] = df.groupby(['X', 'Y']).agg({'Z':'max'})
/var/folders/4v/rx_d3gzj4991_pw5_

In [7]:
gdf_agg.drop(columns=['coords'])

,count,z_median,z_sum,intensity_median,red_median,green_median,blue_median,geometry
ClusterID,,,,,,,,
1,286,6.665223,1904.955737,27.0,33024.0,33280.0,33536.0,"MULTIPOINT ((321498.910 7393115.470), (321499...."
2,382,7.889536,3027.770801,61.0,44288.0,34560.0,31104.0,"MULTIPOINT ((321490.410 7393112.470), (321490...."
3,1079,5.761138,6060.760374,35.0,43520.0,45056.0,45568.0,"MULTIPOINT ((321388.410 7393133.970), (321388...."
4,1445,3.044038,4615.978987,38.0,23040.0,24320.0,25856.0,"MULTIPOINT ((321382.910 7393144.470), (321382...."
5,446,3.831331,1772.834946,58.5,18432.0,15104.0,17152.0,"MULTIPOINT ((321379.410 7393166.970), (321379...."
...,...,...,...,...,...,...,...,...
2906,272,26.186379,7124.084744,78.0,64768.0,64256.0,64768.0,"MULTIPOINT ((322428.410 7393072.470), (322429...."
2907,71,11.874058,843.234607,69.0,63232.0,63488.0,62464.0,"MULTIPOINT ((322428.410 7393085.470), (322427...."
2908,195,5.553757,1086.778086,31.0,13568.0,15616.0,15104.0,"MULTIPOINT ((322441.410 7393060.970), (322441...."


In [22]:
ashapes = gdf_agg.loc[df_agg.loc[:, 'count'] >= 16].coords.apply(lambda x: shapely.concave_hull(MultiPoint(x), ratio=0.1, allow_holes=True))

In [ ]:
geometry = gdf_agg.geometry

In [ ]:
gdf_agg.geometry = ashapes

In [ ]:
gdf_agg.drop(columns=['coords']).to_file(f'results/san_remo-multipoligono.gpkg', driver='GPKG')


In [ ]:
gdf_agg.geometry = geometry